# Total points 8

# In  the homework
- We'll collect climate change related tweet from twitter using REST API
- Use tweets to populate Pandas DataFrames
- Perform some elementary pandas operation to check data sanity


**Please read**

- https://developer.twitter.com/en/developer-terms/more-on-restricted-use-cases


Before proceeding with homework.

<font size="6" color='red'> Create new app at https://apps.twitter.com/app/new  (You may have to create twitter account before this) </font>





[twitter api reference index](https://developer.twitter.com/en/docs/api-reference-index)


# POST oauth2/token
Allows a registered application to obtain an OAuth 2 Bearer Token, which can be used to make API requests on an application's own behalf, without a user context.
https://developer.twitter.com/en/docs/basics/authentication/api-reference/token


Once you have create app, click on app details and keys and token tab

<font size="6" color='red'> write API key value for client_key and API secret key value for client_secret in the next cell</font>

In [41]:
# enter the client key and secret to post for oauth2 token response
client_key = '****************'
client_secret = '*************'

In [42]:
import requests
import base64

key_secret = '{}:{}'.format(client_key, client_secret).encode('ascii')
b64_encoded_key = base64.b64encode(key_secret)
b64_encoded_key = b64_encoded_key.decode('ascii')

In [43]:
# create parameters for REST API post method
base_url = 'https://api.twitter.com/'
auth_endpoint = base_url+'oauth2/token'

auth_headers = { 'Authorization': 'Basic {}'.format(b64_encoded_key),
                'Content-Type': 'application/x-www-form-urlencoded;charset=UTF-8'}

auth_data = { 'grant_type': 'client_credentials'}

Let's post to authentication endpoint and get the token to be used in subsequent REST API call

In [44]:
# call post method and store response (Oauth2 token) in response variable
response = requests.post(auth_endpoint, headers=auth_headers, data=auth_data)
# 200 == success 404 == failure
response.status_code

403

In [45]:
json_data =  response.json()
json_data

{'errors': [{'code': 99,
   'message': 'Unable to verify your credentials',
   'label': 'authenticity_token_error'}]}

In [46]:
access_token = json_data['access_token']
access_token

KeyError: 'access_token'

# Let's do some search on climate change
This is how we got search endpoint

https://developer.twitter.com/en/docs/tweets/search/api-reference/get-search-tweets

In [7]:
max_tweets=10000
tw_limit=100# we can get only 100 tweet per call with standard search api
tweets = []
users = []

In [8]:
# parameters for twitter search API get method
search_headers = {'Authorization': 'Bearer {}'.format(access_token)    
}

parameters = { 'query': 'climate change',
                'max_results': 100,
                'expansions' :'author_id,referenced_tweets.id',
                'tweet.fields' : 'created_at,entities,source,in_reply_to_user_id,geo,lang,public_metrics,reply_settings,referenced_tweets,conversation_id',
                'user.fields': 'created_at,description,entities,id,location,name,pinned_tweet_id,profile_image_url,protected,public_metrics,url,username,verified,withheld'}

search_url = base_url+'2/tweets/search/recent'
# call get mehtod
response = requests.get(search_url, headers=search_headers, params=parameters)

In [9]:
response

<Response [200]>

In [10]:
climate_tweets = response.json()

In [11]:
len(climate_tweets['data'])

100

In [12]:
tweets += climate_tweets['data']
users += climate_tweets['includes']['users']

In [13]:

ids = [tw['id'] for tw in climate_tweets['data']]

In [14]:
min_id = min(ids)

In [15]:
tw_ids = []
tw_ids+=ids

In [16]:
print(type(tw_ids))
print(min(tw_ids))
min_id = min(tw_ids)

<class 'list'>
1517931704891052032


As per twitter api documentation we can use

until_id:Returns results with an ID less than (that is, older than) or equal to the specified ID.

to get older tweets

# Q1 (.5 points) Modify only *parameters* dictionary in the following code so that subsequent call to climate change search returns older tweets with id less than minimum of previous tweets ids.

In [17]:
parameters = { 'query': 'climate change',
                'max_results': tw_limit,
                'expansions' :'author_id,referenced_tweets.id',
                'tweet.fields' : 'created_at,entities,source,in_reply_to_user_id,geo,lang,public_metrics,reply_settings,referenced_tweets,conversation_id',
                'user.fields': 'created_at,description,entities,id,location,name,pinned_tweet_id,profile_image_url,protected,public_metrics,url,username,verified,withheld',
                'until_id': min_id
                 }

for  i in range(max_tweets//tw_limit -1):
    print("searching tweets with id  < {}".format(min_id))
    search_url = base_url+'2/tweets/search/recent'
    response = requests.get(search_url, headers=search_headers, params=parameters)
    climate_tweets = response.json()
    ids = [tw['id'] for tw in climate_tweets['data']]
    tw_ids+=ids
    print(i, len(climate_tweets['data']))
    min_id = min(ids)
    tweets +=climate_tweets['data']
    users +=climate_tweets['includes']['users'] 
    if 'next_token' in response.json()['meta']:
					parameters['next_token'] = response.json()['meta']['next_token']

searching tweets with id  < 1517931704891052032
0 100
searching tweets with id  < 1517931357871407104
1 100
searching tweets with id  < 1517931012377960449
2 100
searching tweets with id  < 1517930747692212226
3 100
searching tweets with id  < 1517930453101076480
4 100
searching tweets with id  < 1517930140889755650
5 100
searching tweets with id  < 1517929839998631936
6 100
searching tweets with id  < 1517929543247482882
7 100
searching tweets with id  < 1517929276087193600
8 100
searching tweets with id  < 1517928974646595584
9 100
searching tweets with id  < 1517928702700507136
10 100
searching tweets with id  < 1517928427311017985
11 100
searching tweets with id  < 1517928163052965889
12 100
searching tweets with id  < 1517927940444635140
13 100
searching tweets with id  < 1517927664694312960
14 100
searching tweets with id  < 1517927323852369922
15 100
searching tweets with id  < 1517927009745207296
16 100
searching tweets with id  < 1517926637144186885
17 100
searching tweets wit

In [19]:
print('Total tweets retrieved= {}'.format(len(tweets)))

Total tweets retrieved= 9995


# Let's print some tweets

In [18]:
import pprint #pretty-print” arbitrary Python data structures. Tweets object has hierarchical struture
max_tweet_to_print=4

In [19]:
pprint.pprint(tweets[0:max_tweet_to_print])

[{'author_id': '23552676',
  'conversation_id': '1517932000870498305',
  'created_at': '2022-04-23T18:22:45.000Z',
  'entities': {'annotations': [{'end': 42,
                                'normalized_text': 'Bill Gates',
                                'probability': 0.9779,
                                'start': 33,
                                'type': 'Person'},
                               {'end': 112,
                                'normalized_text': 'Elon',
                                'probability': 0.9193,
                                'start': 109,
                                'type': 'Person'}],
               'mentions': [{'end': 17,
                             'id': '1263491240336769026',
                             'start': 3,
                             'username': 'WholeMarsBlog'},
                            {'end': 60,
                             'id': '44196397',
                             'start': 51,
                             'username': 'e

Link to understand [tweet-object](https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/tweet-object)

Let'see some tweets

In [20]:
for status in tweets[0:max_tweet_to_print]:
    print(status['text'] + '\n')

RT @WholeMarsBlog: So apparently Bill Gates hit up @elonmusk to discuss “philanthropy on climate change” but Elon asked if he still had a h…

RT @WholeMarsBlog: So apparently Bill Gates hit up @elonmusk to discuss “philanthropy on climate change” but Elon asked if he still had a h…

RT @whatisnuclear: Nuclear fission energy is too slow to help us with climate change you guys. See? https://t.co/1XA37GQ7AC

RT @WholeMarsBlog: So apparently Bill Gates hit up @elonmusk to discuss “philanthropy on climate change” but Elon asked if he still had a h…



# Pandas DataFrame for storing tweet feed

Create following date frame for storing tweets

- **climate_feed_df** DataFrame for storing tweet
- **user_df** DataFrame for storing user detail
- **retweeted_status_df** for storing original tweet

# Printing keys in tweets

In [21]:
print(tweets[1].keys())
print(len(tweets[1].keys()))


dict_keys(['reply_settings', 'source', 'conversation_id', 'public_metrics', 'created_at', 'id', 'entities', 'author_id', 'referenced_tweets', 'lang', 'text'])
11


In [22]:
import pandas as pd

# Generators for building DataFrames

# Q2 (2 point) Complete following   python generator *tweet_generator*. Generator yield modified tweet .
For each tweet in tweets
- Replace **author_id** attribute with **user_id**.
- Replace **referenced_tweets** attribute with **referenced_tweets_ids** and value(s) with **tweet['referenced_tweets']['id']** value(s).
- Keep other attributes in the tweet as such.

In [23]:
def tweet_generator(tweets):
    for tweet in tweets:
      
        modified_tweet= {}
        #Write your code here
        # create deep copy of tweet object
        modified_tweet = tweet.copy()
        modified_tweet['user_id'] = modified_tweet.pop('author_id')
        if 'referenced_tweets' in modified_tweet:
          modified_tweet['referenced_tweets_ids'] = []
          for referenced_tweet in modified_tweet.pop('referenced_tweets'):
            modified_tweet['referenced_tweets_ids'].append(referenced_tweet['id'])
        else:
          modified_tweet['referenced_tweet_ids'] = 'NA'
    
        yield modified_tweet       
            
            

In [24]:
# Checking  generator output
list(tweet_generator(tweets[0:2]))

[{'reply_settings': 'everyone',
  'source': 'Twitter for iPhone',
  'conversation_id': '1517932000870498305',
  'public_metrics': {'retweet_count': 6638,
   'reply_count': 0,
   'like_count': 0,
   'quote_count': 0},
  'created_at': '2022-04-23T18:22:45.000Z',
  'id': '1517932000870498305',
  'entities': {'mentions': [{'start': 3,
     'end': 17,
     'username': 'WholeMarsBlog',
     'id': '1263491240336769026'},
    {'start': 51, 'end': 60, 'username': 'elonmusk', 'id': '44196397'}],
   'annotations': [{'start': 33,
     'end': 42,
     'probability': 0.9779,
     'type': 'Person',
     'normalized_text': 'Bill Gates'},
    {'start': 109,
     'end': 112,
     'probability': 0.9193,
     'type': 'Person',
     'normalized_text': 'Elon'}]},
  'lang': 'en',
  'text': 'RT @WholeMarsBlog: So apparently Bill Gates hit up @elonmusk to discuss “philanthropy on climate change” but Elon asked if he still had a h…',
  'user_id': '23552676',
  'referenced_tweets_ids': ['1517626832241057792']},


In [25]:
# create the dataframe
climate_feed_df = pd.DataFrame(tweet_generator(tweets))

In [26]:
# print for sanity check
climate_feed_df

,reply_settings,source,conversation_id,public_metrics,created_at,id,entities,lang,text,user_id,referenced_tweets_ids,in_reply_to_user_id,referenced_tweet_ids,geo
0,everyone,Twitter for iPhone,1517932000870498305,"{'retweet_count': 6638, 'reply_count': 0, 'lik...",2022-04-23T18:22:45.000Z,1517932000870498305,"{'mentions': [{'start': 3, 'end': 17, 'usernam...",en,RT @WholeMarsBlog: So apparently Bill Gates hi...,23552676,[1517626832241057792],NaN,NaN,NaN
1,everyone,Twitter for iPad,1517931998546911235,"{'retweet_count': 6638, 'reply_count': 0, 'lik...",2022-04-23T18:22:44.000Z,1517931998546911235,"{'mentions': [{'start': 3, 'end': 17, 'usernam...",en,RT @WholeMarsBlog: So apparently Bill Gates hi...,35913027,[1517626832241057792],NaN,NaN,NaN
2,everyone,Twitter Web App,1517931997351616512,"{'retweet_count': 530, 'reply_count': 0, 'like...",2022-04-23T18:22:44.000Z,1517931997351616512,"{'mentions': [{'start': 3, 'end': 17, 'usernam...",en,RT @whatisnuclear: Nuclear fission energy is t...,807882284783304710,[1516244629942726657],NaN,NaN,NaN
3,everyone,Twitter for Android,1517931996802166785,"{'retweet_count': 6638, 'reply_count': 0, 'lik...",2022-04-23T18:22:44.000Z,1517931996802166785,"{'mentions': [{'start': 3, 'end': 17, 'usernam...",en,RT @WholeMarsBlog: So apparently Bill Gates hi...,774316063,[1517626832241057792],NaN,NaN,NaN
4,everyone,Twitter for iPhone,1517344774671454208,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",2022-04-23T18:22:44.000Z,1517931995069886471,"{'mentions': [{'start': 0, 'end': 10, 'usernam...",en,@epic_paco @Tjdriii True but in the end cows p...,1143635721745502210,[1517608242028810241],340125928,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9994,everyone,Twitter for iPhone,1517899656302579712,"{'retweet_count': 2213, 'reply_count': 0, 'lik...",2022-04-23T16:14:13.000Z,1517899656302579712,"{'mentions': [{'start': 3, 'end': 17, 'usernam...",en,RT @VYouthConnect: We are delighted to confer ...,1507652590666121225,[1517110848669421568],NaN,NaN,NaN
9995,everyone,Twitter for Android,1517899654238777351,"{'retweet_count': 5, 'reply_count': 0, 'like_c...",2022-04-23T16:14:13.000Z,1517899654238777351,"{'mentions': [{'start': 3, 'end': 14, 'usernam...",en,RT @10kClimate: Want to be the greatest climat...,1410682350850252803,[1517565179445268486],NaN,NaN,NaN
9996,everyone,Twitter Web App,1517899653844553728,"{'retweet_count': 1653, 'reply_count': 0, 'lik...",2022-04-23T16:14:13.000Z,1517899653844553728,"{'mentions': [{'start': 3, 'end': 15, 'usernam...",en,RT @RickyBeBack: Climate Change aka Global War...,2193047004,[1517608945929494528],NaN,NaN,NaN
9997,everyone,Twitter Web App,1517899648349970434,"{'retweet_count': 1072, 'reply_count': 0, 'lik...",2022-04-23T16:14:12.000Z,1517899648349970434,"{'mentions': [{'start': 3, 'end': 14, 'usernam...",en,"RT @SenSanders: On Earth Day, let us recommit ...",1265563384810033154,[1517565151796420608],NaN,NaN,NaN


# Q3(1 points) List datatype using climate_feed_df. Describe which attributes doesn't have right data types. What should be correct datatype.

In [27]:
# write code here datatype listing
climate_feed_df.dtypes


reply_settings           object
source                   object
conversation_id          object
public_metrics           object
created_at               object
id                       object
entities                 object
lang                     object
text                     object
user_id                  object
referenced_tweets_ids    object
in_reply_to_user_id      object
referenced_tweet_ids     object
geo                      object
dtype: object

# write columns with wrong datatype and mention correct data type too.



- id: Curent Type = object Correct Type = int64
- lang: Current Type = object Correct Type = object or stringDtype extension data type
- text: Current Type = object Correct Type = object or stringDtype extension data type 
- public_metrics: Current Type = object Correct Type = object (note: column contains dictionaries)
- reply_settings: Current Type = object Correct Type = object or stringDtype extension data type
- source: Current Type = object Correct Type = object or stringDtype extension data type
- created_at: Current Type = object Correct Type = datetime64
- conversation_id: Current Tyoe = object Correct Type = int64
- entities: Current Type = object Correct Type = object (note: column contains dictionaries)
- user_id: Current Type = object Correct Type = int64
- referenced_tweets_ids: Current Type = object Correct Type = object (note: column contains list object)
- geo: Current Type = object Correct Type = object (note: column contains geographic data)
- withheld Current Type = object Correct Type = object (note: column contains information about withheld data)

# Q4 (1.5 point) Complete following tweet_user_generator. Generator should yield only *user* value from a tweet

- As per the twitter API the user information is in the 'users' object within the 'includes' object of the response.
- ASIDE: My assignment uses the twitter tweets search API v 2.0. This version of the API returns a different response thus my solution differs from the original assignment. [reference API v 2.0](https://developer.twitter.com/en/docs/twitter-api/tweets/search/api-reference/get-tweets-search-recent#Optional) 

In [28]:
# print users portion of includes response
users

[{'verified': False,
  'description': 'follow me on instagram mike_berg1',
  'location': 'Sacramento',
  'username': 'michaelb2105',
  'profile_image_url': 'https://pbs.twimg.com/profile_images/1419168969139703817/IcqJeylU_normal.jpg',
  'public_metrics': {'followers_count': 81,
   'following_count': 317,
   'tweet_count': 1511,
   'listed_count': 1},
  'id': '23552676',
  'name': 'Mike Berger',
  'created_at': '2009-03-10T03:45:35.000Z',
  'protected': False,
  'url': ''},
 {'verified': False,
  'description': '',
  'location': 'United States',
  'username': 'dandr76437',
  'profile_image_url': 'https://pbs.twimg.com/profile_images/860198669898960897/8hi3sQvQ_normal.jpg',
  'public_metrics': {'followers_count': 91,
   'following_count': 1061,
   'tweet_count': 17605,
   'listed_count': 5},
  'id': '35913027',
  'name': 'Dora Andreas',
  'created_at': '2009-04-27T23:56:52.000Z',
  'protected': False,
  'url': ''},
 {'verified': False,
  'description': 'Cis-hetero-man (he/him/they) Left

In [29]:
""" note: I did not create a generator for this function because using a generator was much less readable and perfomamce did not take a significant hit when using the below
          method for this use case. Below method reads the [includes][users] portion of the response directly inot a dataframe then unpacks columns containing further information.
"""
def tweet_user_generator(tweets):

    users = pd.DataFrame(tweets)
    # unpack column of dictionaries into columns of users dataframe
    users = pd.concat([users, users['public_metrics'].apply(pd.Series)], axis=1)
    users.drop(columns='public_metrics', inplace=True)
    return users

In [30]:
# call tweet generator
user_df = (tweet_user_generator(users))

In [32]:
# print user_df for sanity check
user_df

,verified,description,location,username,profile_image_url,id,name,created_at,protected,url,pinned_tweet_id,entities,followers_count,following_count,tweet_count,listed_count
0,False,follow me on instagram mike_berg1,Sacramento,michaelb2105,https://pbs.twimg.com/profile_images/141916896...,23552676,Mike Berger,2009-03-10T03:45:35.000Z,False,,NaN,NaN,81,317,1511,1
1,False,,United States,dandr76437,https://pbs.twimg.com/profile_images/860198669...,35913027,Dora Andreas,2009-04-27T23:56:52.000Z,False,,NaN,NaN,91,1061,17605,5
2,False,Cis-hetero-man (he/him/they) Left Social Democ...,NaN,Nesslig201,https://pbs.twimg.com/profile_images/138519839...,807882284783304710,Nesslig201,2016-12-11T09:38:41.000Z,False,,1512366615370903556,NaN,98,145,2524,0
3,False,Live from the heart of yourself. Seek to be wh...,Pierrefonds QC,Pumpkanpie,https://pbs.twimg.com/profile_images/797893053...,774316063,Anne(@🏡) LLB,2012-08-22T19:15:04.000Z,False,,NaN,NaN,895,2197,157051,14
4,False,Musician. College student. Hero of the Pine Ba...,"Forked River, NJ",AMusicalTreeRat,https://pbs.twimg.com/profile_images/147907392...,1143635721745502210,Squirrel,2019-06-25T21:42:58.000Z,False,https://t.co/P948sNl4D5,1503914353522786305,"{'url': {'urls': [{'start': 0, 'end': 23, 'url...",34,94,2955,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9708,False,#SAVESOIL,NaN,Laxmanr70120688,https://pbs.twimg.com/profile_images/150795483...,1507652590666121225,Laxman rao #SAVESOIL,2022-03-26T09:36:27.000Z,False,,NaN,"{'description': {'hashtags': [{'start': 0, 'en...",525,642,7590,1
9709,False,"Founder bandani Environmental Champs,wildlife ...","Kisumu, Kenya",MallaykaIanna,https://pbs.twimg.com/profile_images/151749101...,1410682350850252803,Ianna Mallayka Foundation,2021-07-01T19:31:26.000Z,False,,NaN,NaN,1251,1897,303,11
9710,False,"#MAGA, NRA member, Trump supporter, #SaveAmerica","Florida, USA",rspoff2112,https://pbs.twimg.com/profile_images/991809901...,2193047004,Rick Spofford,2013-11-13T22:30:39.000Z,False,,NaN,"{'description': {'hashtags': [{'start': 0, 'en...",2858,2837,87225,0
9711,False,"Remembering habeas corpus bar comment, female ...",NaN,Charles35093463,https://pbs.twimg.com/profile_images/149699929...,1265563384810033154,Sauvignon,2020-05-27T08:46:46.000Z,False,,NaN,NaN,1131,1190,53192,1


# Q5(1 point) List datatype of columns using user_df. Describe which attributes doesn't have right data types. What should be correct datatype.

In [33]:
# write code here for datatype listing
user_df.dtypes

verified               bool
description          object
location             object
username             object
profile_image_url    object
id                   object
name                 object
created_at           object
protected              bool
url                  object
pinned_tweet_id      object
entities             object
followers_count       int64
following_count       int64
tweet_count           int64
listed_count          int64
dtype: object

# write columns with wrong datatype and mention correct data type too.

- created_at: Current Type = object Correct Type = datetime64
- protected: Current Type = boolean Correct Type = boolean
- entities: Current Type = object Correct Type = object (note: column contains dictionaries)
- profile_image_url: Current Type = object Correct Type = object or stringDtype extension data type
- username: Current Type = object Correct Type = object or stringDtype extension data type
- verified: Current Type = boolean Correct type = boolean
- url: Current Type = object Correct Type = object or stringDtype extension data type
- name: Current Type = object Correct Type = object or stringDtype extension data type
- id: Current Type = object Correct Type = int64
- description: Current Type = object Correct Type = object or stringDtype extension data type
- location: Current Type = object Correct Type = object or stringDtype extension data type
- pinned_tweet_id: Current Type = object Correct Type = int64
- withheld Current Type = object Correct Type = object (note: column contains information about withheld data)
- followers_count = Current Type = int64 Correct Type = int64
- following_count = Current Type = int64 Correct Type = int64
- tweet_count = Current Type = int64 Correct Type = int64
- listed_count = Current Type = int64 Correct Type = int64

# Q6 (1 point) Complete following retweeted_status_generator. Generator should yield dictionary of retweeted_status value from each tweet. If a tweet doesn't have this key return empty dictionary.

- As per the twitter API the user information is in the 'users' object within the 'includes' object of the response.
- ASIDE: My assignment uses the twitter tweets search API v 2.0. This version of the API returns a different response thus my solution differs from the original assignment. [reference API v 2.0](https://developer.twitter.com/en/docs/twitter-api/tweets/search/api-reference/get-tweets-search-recent#Optional)

In [36]:
""" 
          note: I did not create a generator for this function because using a generator was much less readable and perfomamce did not take a significant hit when using the below
          method for this use case. Below method reads the [data] portion of the response, pulls the id and author_id for given tweet then unpacks public_metrics to retrieve 
          information regarding retweets for that tweet instance.
"""
from collections import defaultdict
def retweeted_status_generator(tweets):
    # instantiate default dict to store list of values from response
    retweet_dict = defaultdict(list)
    for tweet in tweets:
        retweet_dict['id'].append(tweet['id'])
        retweet_dict['User_id'].append(tweet['author_id'])
        retweet_dict['public_metrics'].append(tweet['public_metrics'])
    # convert to dataframe
    retweet_df = pd.DataFrame(retweet_dict)
    # expand dictionaries in column "public_metrics"
    retweet_df = pd.concat([retweet_df, retweet_df['public_metrics'].apply(pd.Series)], axis=1)
    retweet_df.drop(columns='public_metrics', inplace=True)

    return retweet_df

In [37]:
retweeted_status_df = retweeted_status_generator(tweets)

In [38]:
# retweeted_status_df print for sanity check
retweeted_status_df

,id,User_id,retweet_count,reply_count,like_count,quote_count
0,1517932000870498305,23552676,6638,0,0,0
1,1517931998546911235,35913027,6638,0,0,0
2,1517931997351616512,807882284783304710,530,0,0,0
3,1517931996802166785,774316063,6638,0,0,0
4,1517931995069886471,1143635721745502210,0,0,0,0
...,...,...,...,...,...,...
9994,1517899656302579712,1507652590666121225,2213,0,0,0
9995,1517899654238777351,1410682350850252803,5,0,0,0
9996,1517899653844553728,2193047004,1653,0,0,0
9997,1517899648349970434,1265563384810033154,1072,0,0,0


# Q7 (1 point) Write code to find number of rows in retweeted_status_df with any of the values NaN. print number of such rows.

- Retweet status no longer is returned using the tweets search endpoint. To substitue I'll return tweets with retweet_count = 0 in place of retweet_status = NaN

In [39]:
# Counting retweet_count == 0
retweeted_status_df[retweeted_status_df['retweet_count']==0].head()

,id,User_id,retweet_count,reply_count,like_count,quote_count
4,1517931995069886471,1143635721745502210,0,0,0,0
12,1517931973758574593,1082837902428655616,0,0,0,0
15,1517931970239483905,2329462040,0,0,0,0
17,1517931966288515075,1061901408188989440,0,0,0,0
20,1517931963025346561,1061901408188989440,0,1,0,0


In [40]:
# Write code here
zero_retweets = retweeted_status_df['retweet_count'].value_counts()[0]
print("The number of tweets with 0 retweets is {}.".format(zero_retweets))

The number of tweets with 0 retweets is 2421.
